In [21]:

import sys, csv
import pysindy as ps
import numpy as np
import pandas as pd
from IPython.display import display, Math

In [25]:
# Open csv file
filepath = "../../../../straight.csv"

with open(filepath, 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=' ', quotechar='|')
    df = pd.read_csv(csvfile, sep=r' ', header=0, encoding='ascii', engine='python')

    # Drop all duplicate timestamps
    df.drop_duplicates(inplace=True, subset=["timestamp_ns"])    

    # Subsample step s => only take every s-th entry of the csv file
    s = 20
    df = df.iloc[::s, :]

    # Create numpy array from data
    # Convert ns to seconds
    t = df["timestamp_ns"].to_numpy() / 10e+9
    x = df["pos_x"].to_numpy()
    y = df["pos_y"].to_numpy()
    z = df["pos_z"].to_numpy()

    roll = df["roll"].to_numpy()
    pitch = df["pitch"].to_numpy()
    yaw = df["yaw"].to_numpy()

    lin_vel_x = df["linear_vel_x"].to_numpy()
    lin_vel_y = df["linear_vel_y"].to_numpy()
    lin_vel_z = df["linear_vel_z"].to_numpy()

    ang_vel_x = df["angular_vel_x"].to_numpy()
    ang_vel_y = df["angular_vel_y"].to_numpy()
    ang_vel_z = df["angular_vel_z"].to_numpy()

In [33]:
# Train model
differentiation_method = ps.FiniteDifference(order=2)
polynom = ps.PolynomialLibrary(degree=1)

functions = [lambda x : np.exp(x), lambda x : np.sin(x), lambda x : np.cos(x)]
function_names = [lambda x : "exp("+str(x)+")", lambda x : "sin("+str(x)+")", lambda x : "cos("+str(x)+")"]
lib_custom = ps.CustomLibrary(library_functions=functions, function_names=function_names)

concat_lib = ps.ConcatLibrary([polynom])
optimizer = ps.STLSQ(threshold=0.1)

#feature_names = [feature for feature in df.columns[1:]]
feature_names = ['p_x', 'p_y', 'p_z', r'\phi', r'\theta', r'\psi', 'v_x', 'v_y', 'v_z', r'v_{\phi}', r'v_{\theta}', r'v_{\psi}']

model = ps.SINDy(feature_library=concat_lib,
                differentiation_method=differentiation_method,
                optimizer=optimizer,
                discrete_time=True,
                feature_names=feature_names)

X = np.stack((x, y, z, roll, pitch, yaw, lin_vel_x, lin_vel_y, lin_vel_z, ang_vel_x, ang_vel_y, ang_vel_z), axis=-1)
model.fit(X, t=t)
model.print()

(p_x)[k+1] = 0.999 p_x[k] + 0.019 \psi[k] + 0.029 v_x[k]
(p_y)[k+1] = 1.001 p_y[k]
(p_z)[k+1] = 0.242 1 + 0.622 p_z[k] + -0.015 \phi[k] + -0.049 \theta[k]
(\phi)[k+1] = 1.634 1 + -2.599 p_z[k]
(\theta)[k+1] = 1.538 1 + -2.434 p_z[k]
(\psi)[k+1] = 1.001 \psi[k]
(v_x)[k+1] = -0.030 \theta[k] + -0.019 \psi[k] + 0.998 v_x[k] + 0.106 v_y[k] + -0.076 v_{\phi}[k] + -0.428 v_{\theta}[k] + -0.069 v_{\psi}[k]
(v_y)[k+1] = 0.093 \phi[k] + 0.257 \psi[k] + 0.037 v_x[k] + -0.492 v_y[k] + 0.060 v_z[k] + -0.055 v_{\phi}[k] + -0.043 v_{\theta}[k]
(v_z)[k+1] = -0.206 v_z[k]
(v_{\phi})[k+1] = -7.233 \phi[k] + 9.544 \theta[k] + -0.002 v_x[k] + 0.051 v_y[k] + -0.194 v_z[k] + -0.216 v_{\phi}[k] + 0.019 v_{\theta}[k] + -0.075 v_{\psi}[k]
(v_{\theta})[k+1] = 15.303 \phi[k] + -20.146 \theta[k] + 0.108 v_z[k] + -0.136 v_{\phi}[k] + -0.361 v_{\theta}[k] + -0.108 v_{\psi}[k]
(v_{\psi})[k+1] = 0.848 \phi[k] + -1.153 \theta[k] + 0.039 v_x[k] + -0.073 v_z[k]


In [35]:
# Print LateX legend
legend = {
    'p':"position",
    r'\phi':"roll",
    r'\theta':"pitch",
    r'\psi':"yaw",
    'v':"velocity",
}
legend_str = ""

for k, v in legend.items():
    legend_str += f"{k} : {v}" + r"\\"
print(legend_str)

display(Math(legend_str))
# Print equations in Latex
latex_string = ""
for i in range(len(model.equations())):
    latex_string
    state_var = model.feature_names[i] + "(t+1)"
    eq = model.equations()[i]
    eq = eq.replace("[k]", "(t)")
    eq = eq.replace("+ -", "-")
    latex_string += state_var + r" &= " + eq + r"\\"
latex_string = r"\begin{align}" + latex_string + r"\end{align}"
display(Math(latex_string))


# # Print derivates
# for i in range(len(model.equations())):
#     latex_string
#     state_var = model.feature_names[i]
#     eq = model.equations()[i]
#     eq = eq.replace("[k]", "")
#     eq = eq.replace("+ -", "-")
#     latex_string += r"\dot " + state_var + r" &= " + eq + r"\\"

print(latex_string)

p : position\\\phi : roll\\\theta : pitch\\\psi : yaw\\v : velocity\\


<IPython.core.display.Math object>

<IPython.core.display.Math object>

\begin{align}p_x(t+1) &= 0.999 p_x(t) + 0.019 \psi(t) + 0.029 v_x(t)\\p_y(t+1) &= 1.001 p_y(t)\\p_z(t+1) &= 0.242 1 + 0.622 p_z(t) -0.015 \phi(t) -0.049 \theta(t)\\\phi(t+1) &= 1.634 1 -2.599 p_z(t)\\\theta(t+1) &= 1.538 1 -2.434 p_z(t)\\\psi(t+1) &= 1.001 \psi(t)\\v_x(t+1) &= -0.030 \theta(t) -0.019 \psi(t) + 0.998 v_x(t) + 0.106 v_y(t) -0.076 v_{\phi}(t) -0.428 v_{\theta}(t) -0.069 v_{\psi}(t)\\v_y(t+1) &= 0.093 \phi(t) + 0.257 \psi(t) + 0.037 v_x(t) -0.492 v_y(t) + 0.060 v_z(t) -0.055 v_{\phi}(t) -0.043 v_{\theta}(t)\\v_z(t+1) &= -0.206 v_z(t)\\v_{\phi}(t+1) &= -7.233 \phi(t) + 9.544 \theta(t) -0.002 v_x(t) + 0.051 v_y(t) -0.194 v_z(t) -0.216 v_{\phi}(t) + 0.019 v_{\theta}(t) -0.075 v_{\psi}(t)\\v_{\theta}(t+1) &= 15.303 \phi(t) -20.146 \theta(t) + 0.108 v_z(t) -0.136 v_{\phi}(t) -0.361 v_{\theta}(t) -0.108 v_{\psi}(t)\\v_{\psi}(t+1) &= 0.848 \phi(t) -1.153 \theta(t) + 0.039 v_x(t) -0.073 v_z(t)\\\end{align}
